<!-- NOTEBOOK_METADATA source: "Jupyter Notebook" title: "Agent Evaluation Guide - Pydantic AI" description: "Learn how to evaluate LLM agents with Langfuse: from tracing to automated evaluations. Covers final response, trajectory, and step-level evaluation strategies." category: "Evaluation" -->

# Agent Evaluation Guide

This guide shows how to evaluate LLM-powered agents using [Langfuse](https://langfuse.com) and [Pydantic AI](https://ai.pydantic.dev/). You'll learn three evaluation strategies:

1. **Final Response Evaluation** (black box): Judge the agent's output quality
2. **Trajectory Evaluation** (glass box): Verify the agent took the correct path
3. **Step Evaluation** (white box): Test individual decision points

## What is an Agent?

An agent is an LLM-powered system that operates in a loop:

1. **LLM Call**: Receives input and decides on next action
2. **Action**: Executes tools (e.g., `search_docs()`, `get_overview()`)
3. **Environment**: Tools return results (API responses, data, errors)
4. **Feedback**: LLM receives results and decides next step
5. **Stop**: Loop continues until LLM provides final answer

This entire sequence is a **trace** or **trajectory**.

![Agent loop diagram](https://langfuse.com/images/cookbook/agent-evaluation/agent-loop.png)

## Evaluation Approach

Evaluations catch three failure types:

- **Underspecification**: Agent lacks clear instructions or examples
- **Lack of Understanding**: Developer doesn't know why agent fails
- **Generalization**: LLM fails on new, unseen inputs

### Phased Implementation

1. **Start with Tracing**: Manually inspect traces (15-minute setup, immediate insight)
2. **Add User Feedback**: Capture reactions (thumbs up/down) to flag problematic traces
3. **Build Offline Evals**: Create benchmark dataset for automated testing at scale

This guide focuses on phase 3: automated offline evaluation.

## Step 0: Install Packages

In [ ]:
%pip install -q --upgrade "pydantic-ai[mcp]" langfuse openai nest_asyncio aiohttp

## Step 1: Set Environment Variables

Get your Langfuse API keys from [project settings](https://cloud.langfuse.com).

In [ ]:
import os

os.environ["LANGFUSE_PUBLIC_KEY"] = "pk-lf-..."
os.environ["LANGFUSE_SECRET_KEY"] = "sk-lf-..."
os.environ["LANGFUSE_HOST"] = "https://cloud.langfuse.com"  # EU region
# os.environ["LANGFUSE_HOST"] = "https://us.cloud.langfuse.com"  # US region

os.environ["OPENAI_API_KEY"] = "sk-proj-..."

## Step 2: Enable Langfuse Tracing

Enable automatic tracing for Pydantic AI agents.

In [ ]:
from langfuse import get_client
from pydantic_ai.agent import Agent

langfuse = get_client()
assert langfuse.auth_check(), "Langfuse auth failed - check your keys"

Agent.instrument_all()
print("✅ Pydantic AI instrumentation enabled")

## Step 3: Create Agent

Build an agent that searches Langfuse docs using the [Langfuse Docs MCP Server](https://langfuse.com/docs/docs-mcp).

In [ ]:
from typing import Any
from pydantic_ai import Agent, RunContext
from pydantic_ai.mcp import MCPServerStreamableHTTP, CallToolFunc, ToolResult

LANGFUSE_MCP_URL = "https://langfuse.com/api/mcp"

async def run_agent(item, system_prompt="You are an expert on Langfuse. ", model="openai:gpt-4o-mini"):
    langfuse.update_current_trace(input=item.input)

    tool_call_history = []

    async def process_tool_call(
        ctx: RunContext[Any],
        call_tool: CallToolFunc,
        tool_name: str,
        args: dict[str, Any],
    ) -> ToolResult:
        tool_call_history.append({"tool_name": tool_name, "args": args})
        return await call_tool(tool_name, args)
    
    langfuse_docs_server = MCPServerStreamableHTTP(
        url=LANGFUSE_MCP_URL,
        process_tool_call=process_tool_call,
    )

    agent = Agent(
        model=model,
        system_prompt=system_prompt,
        toolsets=[langfuse_docs_server],
    )

    async with agent:
        result = await agent.run(item.input["question"])
        
        langfuse.update_current_trace(
            output=result.output,
            metadata={"tool_call_history": tool_call_history},
        )

        return result.output, tool_call_history

## Step 4: Create Evaluation Dataset

Build a benchmark dataset with test cases. Each case includes:
- `input`: User question
- `expected_output.response_facts`: Key facts the response must contain
- `expected_output.trajectory`: Expected sequence of tool calls
- `expected_output.search_term`: Expected search query (if applicable)

In [ ]:
test_cases = [
    {
        "input": {"question": "What is Langfuse?"},
        "expected_output": {
            "response_facts": [
                "Open Source LLM Engineering Platform",
                "Product modules: Tracing, Evaluation and Prompt Management"
            ],
            "trajectory": ["getLangfuseOverview"],
        }
    },
    {
        "input": {"question": "How to trace a python application with Langfuse?"},
        "expected_output": {
            "response_facts": [
                "Python SDK, you can use the observe() decorator",
                "Lots of integrations, LangChain, LlamaIndex, Pydantic AI, and many more."
            ],
            "trajectory": ["getLangfuseOverview", "searchLangfuseDocs"],
            "search_term": "Python Tracing"
        }
    },
    {
        "input": {"question": "How to connect to the Langfuse Docs MCP server?"},
        "expected_output": {
            "response_facts": [
                "Connect via the MCP server endpoint: https://langfuse.com/api/mcp",
                "Transport protocol: `streamableHttp`"
            ],
            "trajectory": ["getLangfuseOverview"]
        }
    },
    {
        "input": {"question": "How long are traces retained in langfuse?"},
        "expected_output": {
            "response_facts": [
                "By default, traces are retained indefinitely",
                "You can set custom data retention policy in the project settings"
            ],
            "trajectory": ["getLangfuseOverview", "searchLangfuseDocs"],
            "search_term": "Data retention"
        }
    }
]

DATASET_NAME = "pydantic-ai-mcp-agent-evaluation"

dataset = langfuse.create_dataset(name=DATASET_NAME)
for case in test_cases:
    langfuse.create_dataset_item(
        dataset_name=DATASET_NAME,
        input=case["input"],
        expected_output=case["expected_output"]
    )

## Step 5: Set Up Evaluators

Create three evaluators in Langfuse UI. Each tests a different aspect of agent behavior.

### 1. Final Response Evaluation (Black Box)

Tests output quality. Works regardless of internal implementation.

![Final Response Evaluation](https://langfuse.com/images/cookbook/agent-evaluation/final-response-evaluation.png)

**Prompt template:**

```markdown
You are a teacher grading a student based on the factual correctness of their statements.

### Examples

#### Example 1:
- Response: "The sun is shining brightly."
- Facts to verify: ["The sun is up.", "It is a beautiful day."]
- Reasoning: The response includes both facts.
- Score: 1

#### Example 2:
- Response: "When I was in the kitchen, the dog was there"
- Facts to verify: ["The cat is on the table.", "The dog is in the kitchen."]
- Reasoning: The response mentions the dog but not the cat.
- Score: 0

### New Student Response

- Response: {{response}}
- Facts to verify: {{facts_to_verify}}
```

### 2. Trajectory Evaluation (Glass Box)

Verifies the agent used the correct sequence of tools.

![Trajectory Evaluation](https://langfuse.com/images/cookbook/agent-evaluation/trajectory-evaluation.png)

**Prompt template:**

```markdown
You are comparing two lists of strings. Check whether the lists contain exactly the same items. Order does not matter.

## Examples

Expected: ["searchWeb", "visitWebsite"]
Output: ["searchWeb"]
Reasoning: Output missing "visitWebsite".
Score: 0

Expected: ["drawImage", "visitWebsite", "speak"]
Output: ["visitWebsite", "speak", "drawImage"]
Reasoning: Output matches expected items.
Score: 1

Expected: ["getNews"]
Output: ["getNews", "watchTv"]
Reasoning: Output contains unexpected "watchTv".
Score: 0

## This Exercise

Expected: {{expected}}
Output: {{output}}
```

### 3. Search Quality Evaluation

Validates search query quality when agents search documentation.

**Prompt template:**

```markdown
You are grading whether a student searched for the right information. The search term should correspond vaguely with the expected term.

### Examples

Response: "How can I contact support?"
Expected search topics: Support
Reasoning: Response searches for support.
Score: 1

Response: "Deployment"
Expected search topics: Tracing
Reasoning: Response doesn't match expected topic.
Score: 0

Response: (empty)
Expected search topics: (empty)
Reasoning: No search expected, no search done.
Score: 1

### New Student Response

Response: {{search}}
Expected search topics: {{expected_search_topic}}
```

Create these evaluators in Langfuse UI under **Prompts** → **Create Evaluator**.

## Step 6: Run Experiments

Run agents on your dataset. Compare different models and prompts to find the best configuration.

In [ ]:
dataset = langfuse.get_dataset(DATASET_NAME)

result = dataset.run_experiment(
    name="Production Model Test",
    description="Monthly evaluation of our production model",
    task=run_agent
)

print(result.format())

## Step 7: Compare Multiple Configurations

Test different prompts and models to find the best configuration.

In [ ]:
from functools import partial

system_prompts = {
    "simple": (
        "You are an expert on Langfuse. "
        "Answer user questions accurately and concisely using the available MCP tools. "
        "Cite sources when appropriate."
    ),
    "nudge_search": (
        "You are an expert on Langfuse. "
        "Answer user questions accurately and concisely using the available MCP tools. "
        "Always cite sources when appropriate. "
        "When unsure, use getLangfuseOverview then search the docs. You can use these tools multiple times."
    )
}

models = ["openai:gpt-4o-mini", "openai:gpt-4o"]

dataset = langfuse.get_dataset(DATASET_NAME)

for prompt_name, prompt_content in system_prompts.items():
    for test_model in models:
        task = partial(
            run_agent,
            system_prompt=prompt_content,
            model=test_model,
        )

        result = dataset.run_experiment(
            name=f"Test: {prompt_name} {test_model}",
            description="Comparing prompts and models",
            task=task
        )

        print(result.format())